In [ ]:
import baltic as bt
import requests
from io import StringIO as sio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Polygon, PathPatch
from matplotlib.path import Path
from matplotlib.collections import LineCollection
from matplotlib import cm
import matplotlib.patheffects as path_effects
import numpy as np
from datetime import datetime as dt
import os
import PyAstronomy
from PyAstronomy import pyasl
import datetime
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Polygon
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator
mpl.rcParams['font.family'] = 'sans-serif'
from scipy import stats
import pandas as pd

def get_all_terminals(node):
    terminals = []
    for i in node.children:
        if i.branchType == "leaf":
            terminals.append(i)
        if i.branchType != "leaf":
            terminals.extend(get_all_terminals(i))
    return terminals

treeFile="hMPXV.State.history_MCC.nwk" ## stream from repo copy

ll=bt.loadNexus(treeFile) ## treeFile here can alternatively be a path to a local file
ll.traverse_tree()
ll.treeStats()
ll.sortBranches()

for i in ll.Objects:
    if i.branchType == "leaf":
        if "VSP" in i.name:
            i.traits["rate_signDistribution"]="NEW"

        elif "TRM" in i.name:
            i.traits["rate_signDistribution"]="NEW"
        else:
            i.traits["rate_signDistribution"]="NO"


In [ ]:
df=pd.read_csv("State_class.txt", sep="\t")
mmap=dict(zip(df["taxa"], df["class"]))
mmap.update({'Akwa-Ibom':"S"})
mmap.update({'Cross-River':"S"})
mmap['rivers']="Rivers"

dd={  'Rest': "#C0C0C0",
   "nigeria": "black",
    'NC': "#960018",
  'NE': "#FE5A1D",
  'NW': "#FFD700",
  'SE': "#0070BB",
  'SW': "#138808",
        "Rivers":"#273BE2",

    'S': "#16166B"}


In [ ]:

fig, ax1 = plt.subplots(figsize=(20, 20))
ll.traverse_tree()
ll.sortBranches()
x_attr=lambda k: k.absoluteTime 

x_attr2=[]
for i in ll.Objects:
    if i.branchType == "node":
        if "State.states.prob" in i.traits:
            if i.traits["State.states.prob"] >= 0.80:
                x_attr2.append([i.absoluteTime, i.y, i.traits["posterior"]])
                
for xx in range(2015, 2025, 2):
    plt.axvspan(xx, xx+1, facecolor='#F5F5F5', alpha=0.7, zorder=1, edgecolor="#F5F5F5")        
    
effects=[path_effects.Stroke(linewidth=2, foreground='white'),
                 path_effects.Stroke(linewidth=0.5, foreground='black')]

kwargs={'ha':'left','va':'center','size':8,'path_effects':effects}

text_func=lambda k: k.name
t_target=lambda k: k.branchType=='leaf'

n_target=lambda k: k.branchType=='node' and len(k.children)==1

def c_func(k):
    p=dd[mmap[k.traits["State.states"]]]
    return(p)

s_func=lambda k: 150-30*k.height/ll.treeHeight if k.traits['rate_signDistribution']=='NEW' else 100-30*k.height/ll.treeHeight


ll.plotTree(ax1,x_attr=x_attr,colour=c_func)
ll.plotPoints(ax1,x_attr=x_attr,size=200,colour=c_func,zorder=100) 


plt.xticks([xx for xx in range(2015, 2025, 1)])
plt.yticks([])
plt.tick_params(axis='x', labelsize=30)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
# plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['left'].set_visible(False)

import re
uu=2015
yy=6
for k,v in dd.items():
    k=re.sub("nigeria", "Nigeria", k)
    if k != "Nigeria" and k!= "Rivers"  and k!="Rest":
        k=re.sub("S$", "SS", k)
        plt.scatter(uu,yy,s=1000,color=v,zorder=400, edgecolor='black')
        plt.text(uu+0.2, yy-2, k, size=40)
        # uu=uu+0.005
        yy=yy+10 


plt.scatter(uu,yy,s=1000,color="#273BE2",zorder=400, edgecolor='black')
plt.text(uu+0.2, yy-2, "Rivers", size=40)


plt.savefig("DTA_MCC_state.png",bbox_inches='tight', dpi=1000)   
